# 司法院透過案號抓取檔案
* 要爬取所有資料

## Import & Install

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import math
import re
import enum

In [2]:
# settings
pd.set_option('display.precision', 0)

In [114]:
urls = [
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TPHM%2c112%2c%u91cd%u4e0a%u66f4%u4e8c%2c20%2c20231220%2c1",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TCHM%2c112%2c%e6%8a%97%2c611%2c20230731%2c2",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TPHM%2c110%2c%e9%87%8d%e4%b8%8a%e6%9b%b4%e4%b8%80%2c26%2c20230131%2c1",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TTDV%2c107%2c%e9%87%8d%e8%a8%b4%2c21%2c20211228%2c1",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TPHM%2c107%2c%e4%b8%8a%e8%a8%b4%2c238%2c20200813%2c2",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TPDV%2c105%2c%e8%a8%b4%2c123%2c20200630%2c4",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TPHM%2c107%2c%e4%b8%8a%e8%a8%b4%2c238%2c20200319%2c1",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TYDM%2c107%2c%e6%98%93%2c714%2c20200121%2c1",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=TPDV%2c105%2c%e8%a8%b4%2c123%2c20190222%2c3",
"https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id=NTDM%2c107%2c%e8%a8%b4%2c10%2c20180823%2c1"




]

labels = ['序號','裁判字號','裁判日期','裁判案由','裁判內容','螢光' ,'補充內容']

In [121]:
# use beautiful soup to parse reviews
# 直接找呈現100則評論的頁面
df  = []
for url in urls:
    # Make a GET request to the iframe source URL
    response = requests.get(url, verify=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')
        contents = soup.find('div', class_ = 'int-table col-xs-8')
        tmp = {}

        col_th_elements = soup.find_all('div', class_='col-th')
        col_td_elements = soup.find_all('div', class_='col-td')

        # Iterate over the elements and extract text content
        for col_th, col_td in zip(col_th_elements, col_td_elements):
            col_name = col_th.text.strip()
            col_value = col_td.text.strip()
            tmp[col_name] = col_value

        # content
        content = ''
        contents = soup.find('div', class_ = "jud_content")
        for c in contents.find_all('div'):
            content +=c.text
        tmp['content'] = content
        df.append(tmp)           

    else:
        print("Failed to fetch iframe content:", response.status_code)



In [122]:
tmp_df = pd.DataFrame(df)

In [123]:
tmp_df

,裁判字號：,裁判日期：,裁判案由：,content
0,臺灣高等法院 112 年度重上更二字第 20 號刑事判決,民國 112 年 12 月 20 日,偽造有價證券等,臺灣高等法院刑事判決112年度重上更二字第20號上 訴 人 即 被 告 黃宏裕指...
1,臺灣高等法院 臺中分院 112 年度抗字第 611 號刑事裁定,民國 112 年 07 月 31 日,聲明異議,臺灣高等法院臺中分院刑事裁定 112年度抗字第611號抗告人 即聲明異議人 林君鴻上列抗...
2,臺灣高等法院 110 年度重上更一字第 26 號刑事判決,民國 112 年 01 月 31 日,偽造有價證券等,臺灣高等法院刑事判決 110年度重上更一字第26號上 訴 人 即 被 告 黃宏裕...
3,臺灣臺東地方法院 107 年度重訴字第 21 號民事判決,民國 110 年 12 月 28 日,損害賠償,臺灣臺東地方法院民事判決 ...
4,臺灣高等法院 107 年度上訴字第 238 號刑事判決,民國 109 年 08 月 13 日,偽造有價證券等,臺灣高等法院刑事判決107年度上訴字第238號上 訴 人 即 被 告 黃宏裕選任...
5,臺灣臺北地方法院 105 年度訴字第 123 號民事判決,民國 109 年 06 月 30 日,給付貨款,臺灣臺北地方法院民事判決 ...
6,臺灣高等法院 107 年度上訴字第 238 號刑事裁定,民國 109 年 03 月 19 日,偽造有價證券等,臺灣高等法院刑事裁定107年度上訴字第238號上 訴 人 即 被 告 黃宏裕選任...
7,臺灣桃園地方法院 107 年度易字第 714 號刑事判決,民國 109 年 01 月 21 日,詐欺,臺灣桃園地方法院刑事判決 107年度易字第714號\r\n公 訴 人 臺灣桃園...
8,臺灣臺北地方法院 105 年度訴字第 123 號民事裁定,民國 108 年 02 月 22 日,給付貨款,臺灣臺北地方法院民事裁定 105年度訴字第123號\r\n原 告\r\n即反...
9,臺灣南投地方法院 107 年度訴字第 10 號刑事判決,民國 107 年 08 月 23 日,違反槍砲彈藥刀械管制條例,臺灣南投地方法院刑事判決 107年度訴字第10號\r\n公 訴 人 臺灣南投地...
